In [15]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
import requests
import pandas as pd
import pickle
import os

In [16]:
chaves = {
    'a': 1,
    'b': 2
}

def soma(a, b):
    return a + b

In [17]:
chaves['a']

1

In [18]:
import pendulum

In [19]:
import pendulum
data_hora_atual = pendulum.now('America/Sao_Paulo').to_iso8601_string()
data_hora_atual = pendulum.parse(data_hora_atual)
data_hora_busca = data_hora_atual.subtract(hours=7)
data_hora_busca_formatada = data_hora_busca.format('YYYY_MM_DD_HH_mm_ss')
data = f'extracao_data_{data_hora_busca_formatada}'


In [20]:
data

'extracao_data_2024_07_03_13_46_47'

In [21]:
soma(**chaves)

3

In [22]:
spark = SparkSession.builder.appName('DEPARA').getOrCreate()

In [23]:
df_estatisticas_videos = spark.read.json(
    '../datalake/bronze/cities_skylines/extracao_data_2024_06_29/estatisticas_canais_brasileiros/req_estatisticas_canais_brasileiros.json')

In [24]:
df_estatisticas_videos.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- country: string (nullable = true)
 |    |    |    |-- customUrl: string (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- localized: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- title: string (nullable = true)
 |    |    |    |-- publishedAt: string (nullable = true)
 |    |    |    |-- thumbnails: struct (nullable = true)
 |    |    |    |    |-- default: struct (nullable = true)
 |    |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |    |-- url: string (nullable = true)
 |    

In [25]:
df_estatisticas_videos.select(
    'data_extracao',
    F.explode('items')
).printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- col: struct (nullable = true)
 |    |-- etag: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- kind: string (nullable = true)
 |    |-- snippet: struct (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- customUrl: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- localized: struct (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |    |    |-- publishedAt: string (nullable = true)
 |    |    |-- thumbnails: struct (nullable = true)
 |    |    |    |-- default: struct (nullable = true)
 |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |-- url: string (nullable = true)
 |    |    |    |    |-- width: long (nullable = true)
 |    |    |    |-- high: struct (nullable = true)
 |    |    |    |    |-- height: long (nullable = true)
 |  

In [26]:
def obter_hora(dataframe: DataFrame):
    dataframe = dataframe.withColumn('TURNO_EXTRACAO', F.when((F.col('HORA') >= 0) & (F.col('HORA') < 6), 'Madrugada')
                .when((F.col('HORA') >= 6) & (F.col('HORA') < 12), 'Manhã')
                .when((F.col('HORA') >= 12) & (F.col('HORA') < 18), 'Tarde')
                .otherwise('Noite'))

In [31]:
df_estatisticas_videos = spark.read.json(
    '/home/rodrigo/Documentos/projetos/analise_dados_youtube_segunda_versao/datalake/bronze/cities_skylines/extracao_data_2024_06_29/estatisticas_videos/req_estatisticas_videos.json')

In [32]:
df_estatisticas_videos.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- contentDetails: struct (nullable = true)
 |    |    |    |-- caption: string (nullable = true)
 |    |    |    |-- definition: string (nullable = true)
 |    |    |    |-- dimension: string (nullable = true)
 |    |    |    |-- duration: string (nullable = true)
 |    |    |    |-- licensedContent: boolean (nullable = true)
 |    |    |    |-- projection: string (nullable = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- categoryId: string (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- channelTitle: string (nullable = true)
 |    |    |    |-- defaultAudioLanguage: string (nullable = true)
 |    | 

In [33]:
df_estatisticas_videos.show()

+-------------------+--------------------+--------------------+--------------------+--------+
|      data_extracao|                etag|               items|                kind|pageInfo|
+-------------------+--------------------+--------------------+--------------------+--------+
|2024-06-29 23:34:31|uAyX8MJ2B_udlVdcg...|[{{false, hd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-06-29 23:34:31|10FPEusBbGUKmxKyG...|[{{false, sd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-06-29 23:34:31|_iBTFxkhxH866lRpi...|[{{false, hd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-06-29 23:34:31|cTQwUCzBrNZG08DKD...|[{{false, sd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-06-29 23:37:01|uAyX8MJ2B_udlVdcg...|[{{false, hd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-06-29 23:37:01|10FPEusBbGUKmxKyG...|[{{false, sd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-06-29 23:37:01|_iBTFxkhxH866lRpi...|[{{false, hd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-06-29 23:37:01|cTQwUCzBrNZG08DKD...|[{{false, sd, 2d,.

In [59]:
df_estatisticas_videos.select(F.explode('items')) \
    .printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- contentDetails: struct (nullable = true)
 |    |    |-- caption: string (nullable = true)
 |    |    |-- definition: string (nullable = true)
 |    |    |-- dimension: string (nullable = true)
 |    |    |-- duration: string (nullable = true)
 |    |    |-- licensedContent: boolean (nullable = true)
 |    |    |-- projection: string (nullable = true)
 |    |-- etag: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- kind: string (nullable = true)
 |    |-- snippet: struct (nullable = true)
 |    |    |-- categoryId: string (nullable = true)
 |    |    |-- channelId: string (nullable = true)
 |    |    |-- channelTitle: string (nullable = true)
 |    |    |-- defaultAudioLanguage: string (nullable = true)
 |    |    |-- defaultLanguage: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- liveBroadcastContent: string (nullable = true)
 |    |    |-- localized: struct (nullabl

In [79]:
df_teste = df_estatisticas_videos.select(F.explode('items').alias('items')) \
    .select(
        F.col('items.id').alias('ID_VIDEO'),
        F.col('items.snippet.channelId').alias('ID_CANAL'),
        F.col('items.snippet.title').alias('TITULO_VIDEO'),
        F.col('items.snippet.description').alias('DESCRICAO'),
        F.col('items.contentDetails.duration').alias('DURACAO'),
        F.col('items.snippet.tags').alias('TAGS'),
        
        F.col('items.snippet.categoryid').alias('ID_CATEGORIA'),
        F.col('items.statistics.viewCount').alias('TOTAL_VISUALIZACOES'),
        F.col('items.statistics.likeCount').alias('TOTAL_LIKES'),
        F.col('items.statistics.favoriteCount').alias('TOTAL_FAVORITOS'),
    )
df_teste = df_teste.withColumn('TOTAL_TAGS', F.when(
    F.size(df_teste.TAGS) <= 0, 0).otherwise(F.size(df_teste.TAGS)))
df_teste = df_teste.withColumn('TOTAL_PALAVRAS_TITULO', F.size(F.split(df_teste.TITULO_VIDEO, " ")))
df_teste = df_teste.withColumn(
    'TOTAL_PALAVRAS_DESCRICAO', F.size(F.split(df_teste.DESCRICAO, " ")))
df_teste.show()

+-----------+--------------------+--------------------+--------------------+--------+--------------------+------------+-------------------+-----------+---------------+----------+---------------------+------------------------+
|   ID_VIDEO|            ID_CANAL|        TITULO_VIDEO|           DESCRICAO| DURACAO|                TAGS|ID_CATEGORIA|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_TAGS|TOTAL_PALAVRAS_TITULO|TOTAL_PALAVRAS_DESCRICAO|
+-----------+--------------------+--------------------+--------------------+--------+--------------------+------------+-------------------+-----------+---------------+----------+---------------------+------------------------+
|FudZWNcwonU|UC3OZQD5qIIt27gu_...|Tutorial de como ...|Meta Race\nIP: mt...| PT4M11S|                null|          22|                 59|         16|              0|         0|                   16|                     577|
|RhSZfR7reTM|UCsFvpmOu04Tyf9gs...|Encontrei O Melho...|Slv Tropa Deixa O...| PT1M44S|           

In [80]:
spark.stop()

- DuckDb

In [1]:
import duckdb

In [89]:
import os
lista_pasta = os.listdir('../datalake/prata/cities_skylines/extracao_data_2024_06_29/estatisticas_canais_brasileiros/')
lista_pasta

['req_estatisticas_canais_brasileiros.parquet']